In [1]:
from bs4 import BeautifulSoup
from urllib.request import urlopen
from urllib.request import Request
from selenium import webdriver
from selenium.webdriver.common.by import By
import requests
import time
import pandas as pd
import numpy as np
import os
# import schedule

In [2]:
linkDF=pd.read_excel('./Data/links.xlsx')
linkDF

,Unnamed: 0,상품,링크
0,1,락토핏 골드 2g x 50포,https://cr.shopping.naver.com/adcr.nhn?x=JMFR3...
1,2,덴프스 덴마크 유산균 이야기 170mg x 60캡슐,https://cr.shopping.naver.com/adcr.nhn?x=802aU...
2,3,고려은단비타민C1000 1080mg x 600캡슐,https://search.shopping.naver.com/catalog/5099...
3,4,고려은단 메가도스C 비타민C 3000 3g x 100포,https://cr.shopping.naver.com/adcr.nhn?x=AYDYr...
4,5,주영엔에스 관절엔 콘드로이친 1200 900mg x 60정,https://cr.shopping.naver.com/adcr.nhn?x=dpX%2...
...,...,...,...
117,118,GNM자연의품격 rTG 알티지 오메가3 837mg x 60캡슐,https://cr.shopping.naver.com/adcr.nhn?x=lnIcS...
118,119,종근당건강 홍삼녹용 프리미엄 50ml x 30포,https://cr.shopping.naver.com/adcr.nhn?x=D7uVa...
119,120,에스더포뮬러 여에스더 유산균 골드 500mg x 60캡슐,https://cr.shopping.naver.com/adcr.nhn?x=Vnm22...
120,121,파마젠 알티지 오메가3 1300mg x 180캡슐,https://cr.shopping.naver.com/adcr.nhn?x=gG16a...


In [3]:
# # url = 'https://search.shopping.naver.com/catalog/35054619618?&NaPm=ct%3Dm29on57s%7Cci%3Da8afe02fadc1e3a0b6d6c2b1bc1ca7c4eca4ca02%7Ctr%3Dslcc%7Csn%3D95694%7Chk%3D370e48f40c5bd122d90f0aa114c69681083ba2a7'
# url = linkDF['link_name'][1]

# urlrequest = Request(url, headers={'User-Agent': 'Mozilla/5.0'})

# html = urlopen(urlrequest)
# soup = BeautifulSoup(html, 'html.parser')
# # time.sleep(1)

# ### find 사용

# ### 쇼핑몰 이름 가져오기
# # class : productByMall_mall__SIa50 linkAnchor _nlog_click _nlog_impression_element
# name_tags = soup.find_all('a', class_='productByMall_mall__SIa50')
# # print('div_tags', (div_tags))

# ### 쇼핑몰 가격 가져오기
# # class : linkAnchor _nlog_click _nlog_impression_element
# price_tags = soup.select('td.productByMall_price__MjaUK > a > em')
# # print('div_tags', (div_tags))

# # time.sleep(1)

# ### select 사용
# # div_tags = soup.select('a.productByMall_mall__SIa50')
# # div_tags = soup.select_one('a.productByMall_mall__SIa50').text
# # print('div_tags length: ', len(div_tags))

# ### 쇼핑몰 이름 리스트
# name_list=[]
# for tag in name_tags:
#     div_tag = tag.text.strip()
#     name_list.append(div_tag)
# print(name_list)

# ### 쇼핑몰 가격 리스트
# price_list=[]
# for tag in price_tags:
#     div_tag = tag.text.strip()
#     price_list.append(div_tag)
# print(price_list)

In [4]:
# 폴더 존재 함수
def check_path(path):
    if os.path.exists(path):
        print(f'{path} 경로가 존재.')
    else:
        print(f'{path} 경로가 존재 X, 재설정 요')

In [5]:
PATH = './Data/'
check_path(PATH)

./Data/ 경로가 존재.


In [6]:
# dataDF 만드는 함수 (상점, 가격, 상품명, 시간 형태로 바꾸어줌.)
def makeDF(name_list, price_list):
    dataDF = pd.DataFrame([name_list, price_list])
    dataDF.index=['상점','가격']
    dataDF = dataDF.T
    return dataDF

- 링크와 상품명 가져오기

In [7]:
# url = 'https://search.shopping.naver.com/search/category/100002361?adQuery&catId=50000023&origQuery&pagingIndex=1&pagingSize=40&productSet=total&query&sort=review_rel&timestamp=&viewType=list'

# urlrequest = Request(url, headers={'User-Agent': 'Mozilla/5.0'})

# html = urlopen(urlrequest)
# soup = BeautifulSoup(html, 'html.parser')
# time.sleep(1)

# ### find 사용

# ### 링크 가져오기
# # class : product_link__TrAac linkAnchor _nlog_click _nlog_impression_element
# link_tags = soup.find_all('a', class_='product_link__TrAac')
# print('div_tags length: ', len(link_tags))
# print('div_tags', (link_tags))
# time.sleep(1)


# ### 링크 리스트
# link_list=[]
# name_list=[]
# for tag in link_tags:

#     name_tag = tag.text.strip() # 이름 가져오기
#     name_list.append(name_tag)

#     link_tag = tag['href']
#     link_list.append(link_tag)
# print(name_list)
# print(link_list)

#### 셀레니엄으로 교체

In [8]:
# 스크롤을 내리는 함수 (페이지 끝까지)
def scroll_to_end(driver):
    last_height = driver.execute_script("return document.body.scrollHeight")
    while True:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2) # 페이지가 로드되는 동안 대기
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

In [9]:
# 링크 DF 만드는 함수 (상품, 해당 링크)
def makeLinks(name_list, link_list):
    dataDF = pd.DataFrame([name_list, link_list])
    dataDF.index=['상품','링크']
    dataDF = dataDF.T
    return dataDF

In [10]:
# linkDF 파일 업데이트하기
def updateLink(linkDF):
    if not os.path.isfile('./Data/links.xlsx'):
        linkDF.to_excel('./Data/links.xlsx')
    else:
        baseDF=pd.read_excel('./Data/links.xlsx')
        linkDF=pd.concat([baseDF, linkDF])
        linkDF.drop(['Unnamed: 0'], axis=1, inplace=True) # 'Unnamed: 0' 제거해줘야함.
        linkDF.to_excel('./Data/links.xlsx')
    return linkDF

- 상품들 링크 만들기

In [11]:
# driver = webdriver.Chrome()
# agent_option = webdriver.ChromeOptions()

# user_agent_string = 'Mozilla/5.0'
# agent_option.add_argument('user-agent=' + user_agent_string)

# url = "https://search.shopping.naver.com/search/category/100002361?adQuery=&catId=50000023&origQuery=&pagingIndex=1&pagingSize=80&productSet=total&query=&sort=review_rel&timestamp=&viewType=list"
# driver.get(url)

# for i in range(1,11):

#     driver.implicitly_wait(5)

#     # 스크롤을 최대한 밑으로 내림 (동적 Javascript)
#     scroll_to_end(driver)
#     time.sleep(5)
    
#     driver.find_element(By.CLASS_NAME, 'pagination_next__pZuC6').click() # 다음 클릭하기
#     time.sleep(10)

#     # 스크롤을 최대한 밑으로 내림 (동적 Javascript)
#     scroll_to_end(driver)
#     time.sleep(5)

#     # 페이지 소스 soup로 가져오기
#     html = driver.page_source
#     soup = BeautifulSoup(html, 'html.parser')
#     time.sleep(1)

#     ### 링크 가져오기
#     # X class : product_link__TrAac linkAnchor _nlog_click _nlog_impression_element >> 엉뚱한게 섞여있음
#     # class : product_title__Mmw2K

#     link_tags = soup.select('div.product_title__Mmw2K > a')

#     ### 링크 리스트
#     link_list=[]
#     name_list=[]
#     for tag in link_tags:

#         name_tag = tag.text.strip() # 이름 가져오기
#         name_list.append(name_tag)

#         link_tag = tag['href'] # 링크 가져오기
#         link_list.append(link_tag)

#     print(name_list)
#     print(link_list)

#     linkDF = makeLinks(name_list, link_list)
#     linkDF = updateLink(linkDF)
#     print(f'link 누적개수 : {len(linkDF)}')

# driver.quit()

- 5군데 링크 돌면서 데이터 합치기 (샘플)

In [ ]:
# for idx, link in enumerate(link_list):
#     driver = webdriver.Chrome()
#     agent_option = webdriver.ChromeOptions()

#     user_agent_string = 'Mozilla/5.0'
#     agent_option.add_argument('user-agent=' + user_agent_string)

#     driver.get(link_list[idx])
#     driver.implicitly_wait(5)
#     time.sleep(1)
    
#     html = driver.page_source
#     driver.quit()
#     time.sleep(1)

#     soup = BeautifulSoup(html, 'html.parser')

#     ### 상품 체크
#     print('현재 업데이트할 상품:', name_list[idx])

#     ### 쇼핑몰 이름 가져오기
#     # class : productByMall_mall__SIa50 linkAnchor _nlog_click _nlog_impression_element
#     name_tags = soup.find_all('a', class_='productByMall_mall__SIa50')

#     ### 쇼핑몰 가격 가져오기
#     # class : linkAnchor _nlog_click _nlog_impression_element
#     price_tags = soup.select('td.productByMall_price__MjaUK > a > em')

#     ### 쇼핑몰 이름 리스트
#     name_list=[]
#     for tag in name_tags:
#         name_tag = tag.text.strip()
#         name_list.append(name_tag)
#     print(name_list)

#     ### 쇼핑몰 가격 리스트
#     price_list=[]
#     for tag in price_tags:
#         price_tag = tag.text.strip()
#         price_list.append(price_tag)
#     print(price_list)

#     ### 쇼핑몰 가격, 쇼핑몰 이름 DataFrame에 담아주기 (상점, 가격, 상품명, 시간 형태로 바꾸어줌.)
#     dataDF = makeDF(dataDF)
#     dataDF['상품명'] = name_list[idx]

#     ### 해당 dataDF를 파일에 추가 저장해주고 빠져나감.
#     saveDF=pd.read_excel('./Data/output.xlsx')
    
#     print('업데이트 여부 체크중', len(saveDF)) # 업데이트 여부 체크

#     updateDF = concatFile(saveDF, dataDF)

#     ### 최신본 저장
#     updateDF.to_excel(PATH+"output.xlsx", sheet_name='check')

In [13]:
# driver = webdriver.Chrome()
# agent_option = webdriver.ChromeOptions()

# user_agent_string = 'Mozilla/5.0'
# agent_option.add_argument('user-agent=' + user_agent_string)
# driver.get(link_list[0])
# driver.implicitly_wait(3)

# html = driver.page_source
# driver.quit()

# soup = BeautifulSoup(html, 'html.parser')

# ### 쇼핑몰 이름 가져오기 (find_all 사용)
# # class : productByMall_mall__SIa50 linkAnchor _nlog_click _nlog_impression_element
# name_tags = soup.find_all('a', class_='productByMall_mall__SIa50')
# print('name_tags:', (name_tags))

# # ### 쇼핑몰 가격 가져오기 (select 사용)
# # # class : linkAnchor _nlog_click _nlog_impression_element
# price_tags = soup.select('td.productByMall_price__MjaUK > a > em')
# print('price_tags:', (price_tags))
# # # time.sleep(1)

# # ### 쇼핑몰 이름 리스트
# name_list=[]
# for tag in name_tags:
#     div_tag = tag.text
#     name_list.append(div_tag)
# print(name_list)

# # ### 쇼핑몰 가격 리스트
# price_list=[]
# for tag in price_tags:
#     div_tag = tag.text.strip()
#     price_list.append(div_tag)
# print(price_list)

- links 파일 크롤링

In [14]:
links=pd.read_excel('./Data/links.xlsx')
links

,Unnamed: 0,상품,링크
0,1,락토핏 골드 2g x 50포,https://cr.shopping.naver.com/adcr.nhn?x=JMFR3...
1,2,덴프스 덴마크 유산균 이야기 170mg x 60캡슐,https://cr.shopping.naver.com/adcr.nhn?x=802aU...
2,3,고려은단비타민C1000 1080mg x 600캡슐,https://search.shopping.naver.com/catalog/5099...
3,4,고려은단 메가도스C 비타민C 3000 3g x 100포,https://cr.shopping.naver.com/adcr.nhn?x=AYDYr...
4,5,주영엔에스 관절엔 콘드로이친 1200 900mg x 60정,https://cr.shopping.naver.com/adcr.nhn?x=dpX%2...
...,...,...,...
117,118,GNM자연의품격 rTG 알티지 오메가3 837mg x 60캡슐,https://cr.shopping.naver.com/adcr.nhn?x=lnIcS...
118,119,종근당건강 홍삼녹용 프리미엄 50ml x 30포,https://cr.shopping.naver.com/adcr.nhn?x=D7uVa...
119,120,에스더포뮬러 여에스더 유산균 골드 500mg x 60캡슐,https://cr.shopping.naver.com/adcr.nhn?x=Vnm22...
120,121,파마젠 알티지 오메가3 1300mg x 180캡슐,https://cr.shopping.naver.com/adcr.nhn?x=gG16a...


In [15]:
PATH = './Data/'
check_path(PATH)

./Data/ 경로가 존재.


In [16]:
FILE = 'savefile.xlsx'

In [17]:
# 파일 업데이트 함수
def updateFile(dataDF):
    if not os.path.isfile(PATH+FILE):
        dataDF.to_excel(PATH+FILE, sheet_name='rowdata')
    else:
        baseDF=pd.read_excel(PATH+FILE)
        dataDF=pd.concat([baseDF, dataDF])
        dataDF.drop(['Unnamed: 0'], axis=1, inplace=True) # 'Unnamed: 0' 제거해줘야함.
        dataDF.to_excel(PATH+FILE, sheet_name='rowdata')
    return dataDF

In [18]:
# 크롤링 함수
def crawling(links):
    for idx, link in enumerate(links['링크']):
        driver = webdriver.Chrome()
        agent_option = webdriver.ChromeOptions()

        user_agent_string = 'Mozilla/5.0'
        agent_option.add_argument('user-agent=' + user_agent_string)

        driver.get(links['링크'][idx])
        driver.implicitly_wait(5)
        time.sleep(1)
        
        html = driver.page_source
        driver.quit()
        time.sleep(1)

        soup = BeautifulSoup(html, 'html.parser')

        ### 상품 체크
        print('현재 업데이트할 상품:', links['상품'][idx])

        ### 쇼핑몰 이름 가져오기
        # class : productByMall_mall__SIa50 linkAnchor _nlog_click _nlog_impression_element
        name_tags = soup.find_all('a', class_='productByMall_mall__SIa50')

        ### 쇼핑몰 가격 가져오기
        # class : linkAnchor _nlog_click _nlog_impression_element
        price_tags = soup.select('td.productByMall_price__MjaUK > a > em')

        ### 쇼핑몰 이름 리스트
        name_list=[]
        for tag in name_tags:
            name_tag = tag.text.strip()
            name_list.append(name_tag)
        print(name_list)

        ### 쇼핑몰 가격 리스트
        price_list=[]
        for tag in price_tags:
            price_tag = tag.text.strip()
            price_list.append(price_tag)
        print(price_list)

        ### 쇼핑몰 가격, 쇼핑몰 이름 DataFrame에 담아주기 (상점, 가격, 상품명, 시간 형태로 바꾸어줌.)
        dataDF = makeDF(name_list, price_list)
        dataDF['상품명'] = links['상품'][idx]
        dataDF['시간'] = time.strftime('%m월 %d일 %H시 %M분')

        ### 파일에 업데이트하고 덮어쓰기
        saveDF = updateFile(dataDF)
        print('업데이트 여부 체크중', len(saveDF)) # 업데이트 여부 체크

In [22]:
# 테스트용 데이터
crawling(links)

현재 업데이트할 상품: 락토핏 골드 2g x 50포
['프리미엄데일리', '', 'A 에이든', '매드스토어', '유씨네건기식', '칠평상사', '', '디카라이프 주식회사', '명품코스메틱1', '케이엔컴퍼니']
['8,070', '8,080', '8,090', '8,370', '8,370', '8,370', '8,390', '8,400', '8,770', '8,790']
업데이트 여부 체크중 119030
현재 업데이트할 상품: 덴프스 덴마크 유산균 이야기 170mg x 60캡슐
['프리미엄데일리', '밸런스', 'jbmall', '', '디카라이프 주식회사', '홈쇼핑앤샵', '주식회사 그린데이', '코스트코 당일배송', '파란고래', '건남유통']
['22,590', '22,600', '22,600', '22,850', '22,900', '22,920', '22,960', '23,490', '23,500', '23,500']
업데이트 여부 체크중 119040
현재 업데이트할 상품: 고려은단비타민C1000 1080mg x 600캡슐
['프리미엄데일리', '', '고려은단비타민C', '', 'NS홈쇼핑', '건강비밀', '', '멸치쇼핑', '', '']
['42,850', '42,860', '42,900', '42,990', '43,300', '43,900', '44,490', '44,500', '44,900', '45,000']
업데이트 여부 체크중 119050
현재 업데이트할 상품: 고려은단 메가도스C 비타민C 3000 3g x 100포
['고려은단몰', '부르소마', '', '웰리치버디', '데일리Shop', 'MJCO', '내몸백서', '오핑마켓', '건강프로샵', '']
['18,900', '26,290', '26,300', '28,500', '28,570', '28,890', '28,900', '29,100', '29,100', '29,250']
업데이트 여부 체크중 119060
현재 업데이트할 상품: 주영엔에스 관절엔 콘드로이친 1200 900

In [21]:
# 크롤링 스케쥴링
# while True:
#     # schedule.every(5).minutes.do(crawling(links['링크']))
#     crawling(links)
#     time.sleep(10)
#     if int(time.strftime('%H')) >= 7:
#         break

현재 업데이트할 상품: 락토핏 골드 2g x 50포
['더건강한몰', '프리미엄데일리', '이셀러커머스', '유씨네건기식', 'A 에이든', '매드스토어', '칠평상사', '', '', '디카라이프 주식회사']
['8,100', '8,160', '8,210', '8,220', '8,240', '8,370', '8,370', '8,380', '8,390', '8,400']
업데이트 여부 체크중 103760
현재 업데이트할 상품: 덴프스 덴마크 유산균 이야기 170mg x 60캡슐
['더건강한몰', '이음건강', '', '프리미엄데일리', '밸런스', 'jbmall', '', '홈쇼핑앤샵', '디카라이프 주식회사', '주식회사 그린데이']
['20,830', '20,900', '22,200', '22,510', '22,600', '22,600', '22,630', '23,090', '23,100', '23,140']
업데이트 여부 체크중 103770
현재 업데이트할 상품: 고려은단비타민C1000 1080mg x 600캡슐
['올리브앤파인', '더건강한몰', '프리미엄데일리', '고려은단비타민C', '', 'NS홈쇼핑', '건강비밀', '', '멸치쇼핑', '']
['42,340', '42,350', '42,400', '42,900', '42,990', '43,300', '43,900', '44,490', '44,500', '44,680']
업데이트 여부 체크중 103780
현재 업데이트할 상품: 고려은단 메가도스C 비타민C 3000 3g x 100포
['더건강한몰', '프리미엄데일리', '고려은단몰', '', '', '부르소마', '웰리치버디', '데일리Shop', 'MJCO', '내몸백서']
['25,730', '25,800', '25,900', '26,370', '27,200', '28,400', '28,500', '28,570', '28,890', '28,900']
업데이트 여부 체크중 103790
현재 업데이트할 상품: 주영엔에스 관절엔 콘드로이친 1200